In [1]:
from opendataval.dataloader import DataFetcher

In [2]:
from opendataval.dataloader import DataFetcher

DataFetcher.datasets_available()  # ['dataset_name1', 'dataset_name2']

{'2dplanes',
 'MiniBooNE',
 'adult',
 'bbc',
 'bbc-embeddings',
 'breast_cancer',
 'challenge-iris',
 'cifar10',
 'cifar10-embeddings',
 'cifar10-val',
 'cifar10-val-embeddings',
 'cifar100',
 'cifar100-val',
 'cifar100-val-embeddings',
 'creditcard',
 'diabetes',
 'digits',
 'echoMonths',
 'election',
 'electricity',
 'fashion',
 'fried',
 'gaussian_classifier',
 'gaussian_classifier_high_dim',
 'imagenet-val',
 'imagenet-val-embeddings',
 'imdb',
 'imdb-embeddings',
 'iris',
 'linnerud',
 'lowbwt',
 'mnist',
 'mv',
 'nomao',
 'pol',
 'stl10-embeddings',
 'stock',
 'svhn-embeddings',
 'wave_energy'}

In [3]:
fetcher = DataFetcher(dataset_name='mnist')

fetcher = fetcher.split_dataset_by_count(2000, 2000, 0)

In [4]:
from opendataval.dataloader.noisify import * 

In [157]:
x_train, y_train, x_valid, y_valid, x_test, y_test = fetcher.datapoints

In [196]:
# mix_labels(fetcher, noise_rate =.3)
data_dict = add_gauss_noise(fetcher, noise_rate =.3)

In [197]:
def values(dual_sol, training_size):
    dualsol = dual_sol

    f1k = np.array(dualsol.squeeze())

    trainGradient = [0] * training_size
    trainGradient = (1 + 1 / (training_size - 1)) * f1k - sum(f1k) / (training_size - 1)
    return list(trainGradient)

In [198]:
class DatasetSplit(Dataset):

    def __init__(self, data_with_labels,numpy_data,label):
        self.data_with_labels = data_with_labels
        self.dataset = torch.Tensor(numpy_data)
        self.targets = torch.LongTensor(label)
        self.idxs = np.array([i for i in range(len(label))])
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        image, label = self.data_with_labels[index]
        return image, label

In [199]:
import torch
from torch.utils.data import Dataset, DataLoader
from otdd.pytorch.moments import *
from otdd.pytorch.utils import *


def process_data(x,y):
    index_list = torch.argmax(torch.Tensor(y), dim=1)


    index_list = np.array(index_list.tolist())

    numpy_data = x
    numpy_labels = index_list 

    assert len(numpy_data) == len(numpy_labels)

    data_with_labels = [(data, label) for data, label in zip(numpy_data, numpy_labels)]

    batch_size = 32  
    dataset = DatasetSplit(data_with_labels,numpy_data,numpy_labels)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    targets1 = dataset.targets
    vals1, cts1 = torch.unique(targets1, return_counts=True)
    min_labelcount = 2
    V1 = torch.sort(vals1[cts1 >= min_labelcount])[0]
    idxs1 = np.array([i for i in range(len(targets1))])
    classes1 = vals1
    
    M1, C1 = compute_label_stats(data_loader, targets1, idxs1, classes1, diagonal_cov=True)
    # print(M1.shape)
    # print(C1.shape)
    DA = (dataset.dataset.view(-1,784).to('cuda'), dataset.targets.to('cuda'))
    XA = augmented_dataset(DA, means=M1, covs=C1, maxn=10000)
    
    return XA

In [200]:
XA = process_data(data_dict['x_train'],y_train)

in MOMENTS:  False


In [201]:
XT = process_data(data_dict['x_valid'],y_valid)

in MOMENTS:  False


In [202]:
XA

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.5418,  0.5391,  0.5318],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5418,  0.5391,  0.5318],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5418,  0.5391,  0.5318],
        ...,
        [ 1.7831,  0.5955, -0.2589,  ...,  0.5418,  0.5391,  0.5318],
        [ 0.3625, -0.6955, -0.2475,  ...,  0.5418,  0.5391,  0.5318],
        [-0.2277, -0.1224, -1.9400,  ...,  0.5418,  0.5391,  0.5318]],
       device='cuda:0')

In [203]:
XT

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.5147,  0.5615,  0.5308],
        [-0.3109,  0.6834, -0.4848,  ...,  0.5147,  0.5615,  0.5308],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5147,  0.5615,  0.5308],
        ...,
        [-0.6672, -1.8473,  1.2069,  ...,  0.5147,  0.5615,  0.5308],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5147,  0.5615,  0.5308],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5147,  0.5615,  0.5308]],
       device='cuda:0')

In [204]:
noises = data_dict['noisy_train_indices']
noises

array([1276,  331, 1997, 1958,  205, 1202,  608, 1413, 1538,  440, 1523,
       1226,  962, 1076, 1764,  456,  763, 1525, 1023,  756, 1814,  644,
        725, 1934, 1405,  812,  406, 1965, 1606,  931, 1471,  186,  305,
        281, 1796, 1218, 1507, 1306, 1737, 1185,   38,  570, 1563,  488,
        323,  575,  548, 1396,  534, 1366, 1527, 1869, 1186,  479, 1672,
       1805,  889,  938,  755, 1902, 1597,  697,  544, 1983, 1312,   45,
       1327, 1879,  580, 1106,  903,  636,  797, 1959,  743,   34,  713,
       1489,   39, 1920,  629, 1631,  609, 1007, 1845, 1304, 1616, 1623,
       1953,  231,  516, 1158, 1157,  482,  794,  883,  899,  767, 1830,
       1533, 1996, 1600,  867, 1749, 1729, 1169,  845,  411, 1711, 1582,
       1588,  258,  949,   78, 1041,  153,  541,  679,  628, 1946,  421,
       1115, 1581, 1873,  830,  728, 1544,  261, 1475, 1275,  437,  989,
       1894,  654,  752, 1361, 1315, 1133,  318,  221,  144, 1858, 1256,
        614, 1194,  552, 1402,  813,  981,  641,  9

In [205]:
import geomloss
loss = geomloss.SamplesLoss(
        loss='sinkhorn', p=2,
        # cost=cost_geomloss,
        debias=True,
        blur=0.1 ** (1 / 2),
        backend='tensorized',
        potentials=True
    )

In [206]:
from FedLava import FedOT
n_supp = 2000
n_epoch = 10
t_val = 0.5
XA = XA.cpu()
XT = XT.cpu()
fedot_pt1 = FedOT(n_supp=n_supp, n_epoch=n_epoch, metric='sqeuclidean', t_val=t_val)
fedot_pt1 = fedot_pt1.fit(XA, XT.requires_grad_(True))
int_meas1 = fedot_pt1.int_meas


0-th epoch the cost is 107.85235780631534
1-th epoch the cost is 57.638281508737435
2-th epoch the cost is 35.279750492097406
3-th epoch the cost is 26.930019646231898
4-th epoch the cost is 24.396924209176895
5-th epoch the cost is 23.71354289221463
6-th epoch the cost is 23.526080325584374
7-th epoch the cost is 23.469092539856785
8-th epoch the cost is 23.454330251443302
9-th epoch the cost is 23.45063822732041


In [207]:
int_meas1 = fedot_pt1.int_meas
t_int_meas1= torch.from_numpy(int_meas1)
dual1 = np.array(loss(XA.double(), t_int_meas1.double())[0])

value = values( torch.from_numpy(dual1), len(XA))
counts = np.where( np.array(value)>0)[0]
k=0
for item in counts:
    if item in noises:
        k +=1
result_1 = k/len(noises)
count1 = k

In [208]:
result_1

1.0

In [209]:
data_dict = mix_labels(fetcher, noise_rate =.3)
data_dict.keys()

dict_keys(['y_train', 'y_valid', 'noisy_train_indices'])

In [210]:
index_list = torch.argmax(torch.Tensor(data_dict['y_train']), dim=1)
train = np.array( [ x_train[i].numpy() for i in range(len(x_train))] )
# numpy_data = x
numpy_labels = index_list 
XA = process_data(train,data_dict['y_train'])


in MOMENTS:  False


In [211]:
index_list = torch.argmax(torch.Tensor(data_dict['y_train']), dim=1)
valid = np.array( [ x_valid[i].numpy() for i in range(len(x_valid))] )
# numpy_data = x
numpy_labels = index_list 
XT = process_data(valid,data_dict['y_valid'])

in MOMENTS:  False


In [212]:
from FedLava import FedOT
n_supp = 2000
n_epoch = 10
t_val = 0.5
XA = XA.cpu()
XT = XT.cpu()
fedot_pt1 = FedOT(n_supp=n_supp, n_epoch=n_epoch, metric='sqeuclidean', t_val=t_val)
fedot_pt1 = fedot_pt1.fit(XA, XT.requires_grad_(True))
int_meas1 = fedot_pt1.int_meas

0-th epoch the cost is 99.68082302305707
1-th epoch the cost is 50.36807803704703
2-th epoch the cost is 26.212830935509697
3-th epoch the cost is 14.988441611551803
4-th epoch the cost is 10.440992024810804
5-th epoch the cost is 8.916624862704452
6-th epoch the cost is 8.377997269697893
7-th epoch the cost is 8.21125070793787
8-th epoch the cost is 8.16898919111248
9-th epoch the cost is 8.158389606323155


In [213]:
int_meas1 = fedot_pt1.int_meas
t_int_meas1= torch.from_numpy(int_meas1)
dual1 = np.array(loss(XA.double(), t_int_meas1.double())[0])

value = values( torch.from_numpy(dual1), len(XA))
counts = np.where( np.array(value)>0)[0]
k=0
for item in counts:
    if item in noises:
        k +=1
result_1 = k/len(noises)
count1 = k

print(result_1)
print(count1)

0.4583333333333333
275


In [214]:
# If access data samples 
dual2 = np.array(loss(XA.double(),XT.double())[0])

value = values( torch.from_numpy(dual2), len(XA))
counts = np.where( np.array(value)>0)[0]
k=0
for item in counts:
    if item in noises:
        k +=1
result_1 = k/len(noises)
count1 = k

print(result_1)
print(count1)

0.4666666666666667
280
